# Multiagent + Similar Search （不保证答案正确）

In [1]:
from functions import * 
from multiagent_func import *
import json
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.output_parsers import StrOutputParser
import asyncio
import nest_asyncio
import openai

nest_asyncio.apply()

In [2]:
with open("test_problem_set.json", 'r') as file:
    problem_set = json.load(file)

problems = []
answers = []
for i in range(len(problem_set)):
    problems.append(problem_set[i]['problem'])
    answers.append(problem_set[i]['answer'])

In [3]:
Notes = []
Explanations = []
correct = 0
is_true = [0 for _ in range(100)]

In [4]:
async def check(note1: str, note2: str) -> bool:
    llm = ChatOpenAI(model='gpt-4o')
    sys_prompt =    r"""
                        你是一个题解检查者，你负责检查两个题解的最终答案是否一致。注意：题解不需要一模一样，
                        只要保证最后每一问的答案都完全一致即可。
                        如果答案一致：请输出是。如果答案不一致：请输出否。请确保输出一个字，不要输出“是“与”否”意外的内容

                        题解1:
                        {note1}
                        
                        题解2:
                        {note2}
                    """
    prompt = ChatPromptTemplate.from_template(sys_prompt)
    parser = StrOutputParser()
    chain = prompt | llm | parser

    response = chain.invoke(input={"note1": note1, "note2": note2})
    return response

In [5]:
await check("算术平方根等于它本身的数是 0 或 1。", "故答案是 $0$ 和 $1$。")

'是'

In [6]:
# %%capture cap --no-stderr
l,r = 60, 80
for i in range(l, r):
    try:
        sim_problems = await search_similar(problems[i])
        cur_note, cur_expl = run(problems[i], sim_problems, "大学四年级")
        Notes.append(cur_note)
        Explanations.append(cur_expl)
        consistency = await check(cur_note, answers[i])
    except Exception as e:
        print(e)
        Notes.append("this is an error")
        Explanations.append("this is an error")
        consistency = "否"
    if consistency == "是":
        correct += 1
        is_true[i] = 1
print(correct / 10)

# with open('outputs/logs.txt', 'w') as f:
#     f.write(cap.stdout)

executing
1. 数学领域和类型: 代数
2. 核心概念和原理: 速度、时间、距离、区间
3. 解题方法和策略: 代入法、不等式求解
4. 问题背景和情境: 交通问题
5. 变量数量: 1
6. 题目所属年级: 8{'Supervisor': {'next': 'Noter'}}
----
{'Noter': {'messages': [HumanMessage(content='知识点讲解:本题考查的知识点是速度、时间和距离之间的关系，以及不等式的解法，解题关键点是利用公式\\(时间 = 距离 / 速度\\)建立不等式并求解。\n\n### 板书一:\n由题意，去程的距离为：\n\\( 距离 = 75 \\, \\text{km/h} \\times 2 \\, \\text{h} = 150 \\, \\text{km} \\)\n\n### 板书二:\n返回时的速度范围为：\n\\( 50 \\, \\text{km/h} \\leq \\text{速度} \\leq 60 \\, \\text{km/h} \\)\n\n### 板书三:\n返回所需时间的取值范围为：\n\\( \\frac{150 \\, \\text{km}}{60 \\, \\text{km/h}} \\leq \\text{时间} \\leq \\frac{150 \\, \\text{km}}{50 \\, \\text{km/h}} \\)\n\n### 板书四:\n解得：\n\\( 2.5 \\, \\text{h} \\leq \\text{时间} \\leq 3 \\, \\text{h} \\)\n\n故答案是2.5小时到3小时。', name='Noter')]}}
----
{'Supervisor': {'next': 'Validator'}}
----
{'Validator': {'messages': [HumanMessage(content='Noter使用的解题方法包括：\n\n1. 使用速度、时间和距离之间的关系公式：\\(时间 = 距离 / 速度\\)。\n2. 建立不等式并求解。\n\n经过检查，这些解题方法对于大学四年级的学生来说并不超纲。\n\n不需要修改。', name='Validator')]}}
----
{'Superviso

Python REPL can execute arbitrary code. Use with caution.


{'Noter': {'messages': [HumanMessage(content='知识点讲解:本题考查的知识点是平方根、立方根和绝对值的计算，解题关键点是正确应用这些基本运算。\n\n### 板书一:\n(1)\n√16 - √3 + 3√8\n= 4 - √3 + 3 * 2√2\n= 4 - √3 + 6√2\n≈ 10.753230566669695\n\n### 板书二:\n(2)\n√9 + 3√(-125) + |√3 - 2|\n= 3 + 3 * (-5) + |√3 - 2|\n= 3 - 15 + |√3 - 2|\n= 3 - 15 + |1.7320508075688772 - 2|\n= 3 - 15 + | -0.2679491924311228 |\n= 3 - 15 + 0.2679491924311228\n≈ -11.732050807568873\n\n故答案是-11.732050807568873。', name='Noter')]}}
----
{'Supervisor': {'next': 'Validator'}}
----
{'Validator': {'messages': [HumanMessage(content='### 解题方法:\n1. 平方根的计算\n2. 立方根的计算\n3. 绝对值的计算\n\n经过对比，平方根、立方根和绝对值的计算都是大学四年级学生应当掌握的基本运算。\n\n不需要修改', name='Validator')]}}
----
{'Supervisor': {'next': 'Explainer'}}
----
Error code: 400 - {'error': {'message': "This model's maximum context length is 128000 tokens. However, your messages resulted in 419053 tokens (418977 in the messages, 76 in the functions). Please reduce the length of the messages or functions.", 'type': 'invalid_request_error', 'param

In [7]:
for i in range(len(Notes)):
    if isinstance(Notes[i], openai.BadRequestError):
        Notes[i] = 'error'
        Explanations[i] = 'error'
    # print(type(Notes[i]))

In [8]:
with open("outputs/model_1_output.json", 'r') as f:
    data = json.load(f)
print(data)
for i in range(len(Notes)):
    cur = {}
    cur['problem'] = problems[l+i]
    cur['notes'] = Notes[i]
    cur['explanation'] = Explanations[i]
    data.append(cur)
print(data)
with open("outputs/model_1_output.json", 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

[{'problem': '把下列各数填入表示它所在的数集的大括号中：\n\n-2.4, π, 2.022, -10/3, -0.15, 0, -√9, -10, -1.101 001 000 1⋯。\n\n(1) 整数集合：{ … }；\n\n(2) 正实数集合：{ … }；\n\n(3) 无理数集合：{ … }。', 'notes': '知识点讲解: 本题考查的知识点是数的分类，解题关键点是根据数的性质将其分类到不同的数集。\n\n### 板书一:\n\n(1) 整数集合：\n\\[\n-√9, -10, 0\n\\]\n\n### 板书二:\n\n(2) 正实数集合：\n\\[\n2.022\n\\]\n\n### 板书三:\n\n(3) 无理数集合：\n\\[\nπ, -1.1010010001\\ldots\n\\]\n\n故答案是：\n(1) 整数集合：{ -√9, -10, 0 }；\n(2) 正实数集合：{ 2.022 }；\n(3) 无理数集合：{ π, -1.1010010001⋯ }。', 'explanation': '### 讲解一:\n\n我们来一步一步分析这些数。\n\n首先是整数集合。整数包括负整数、零和正整数。因此我们需要找出所有符合这个条件的数。\n\n\\[\n-10, 0, -√9\n\\]\n\n这里，-10 和 0 是显而易见的整数。-√9 其实是 -3，也是一个整数。\n\n所以，整数集合是：\n\n\\[\n\\{-10, 0, -√9\\}\n\\]\n\n### 讲解二:\n\n接下来是正实数集合。正实数是大于零的数。\n\n\\[\n2.022\n\\]\n\n在给定的数中，只有 2.022 是大于零的正实数。\n\n所以，正实数集合是：\n\n\\[\n\\{2.022\\}\n\\]\n\n### 讲解三:\n\n最后是无理数集合。无理数是不能表示为两个整数之比的小数，比如 π 和一些非循环小数。\n\n\\[\nπ, -1.1010010001\\ldots\n\\]\n\n在给定的数中，π 和 -1.101 001 000 1⋯ 是无理数。\n\n所以，无理数集合是：\n\n\\[\n\\{π, -1.1010010001\\ldots\\}\n\\]\n\n综上所述，答案是：\n(1) 整数集合：{ -

In [9]:
print(Notes[19])
print(len(data))

this is an error
80
